In [1]:
import json
import glow
glow.register(spark)

In [7]:
df = spark.read.format("text").load('file:///mnt/SAIGE/extdata/output/example_binary.varianceRatio.txt.sparseGRM.mtx.sample')

In [9]:
df.count()

1000

In [37]:
saige_script = r"""#!/bin/sh
set -e

export tmpdir=$(mktemp -d -t vcf.XXXXXX)
cat - > ${tmpdir}/input.vcf
bgzip -c ${tmpdir}/input.vcf > ${tmpdir}/input.vcf.gz
tabix -p vcf ${tmpdir}/input.vcf.gz

export dataDir='/mnt/SAIGE/extdata'
 
/opt/conda/default/bin/Rscript /mnt/SAIGE/extdata/step2_SPAtests.R \
    --vcfFile=${tmpdir}/input.vcf.gz \
    --vcfFileIndex=${tmpdir}/input.vcf.gz.tbi \
    --vcfField=DS \
    --chrom=1 \
    --minMAF=0.0001 \
    --minMAC=1 \
    --GMMATmodelFile=${dataDir}/output/example_binary.rda \
    --varianceRatioFile=${dataDir}/output/example_binary.varianceRatio.txt \
    --SAIGEOutputFile=${tmpdir}/output.txt \
    --numLinesOutput=2 \
    --IsOutputAFinCaseCtrl=TRUE \
    --IsOutputNinCaseCtrl=TRUE \
    --IsOutputHetHomCountsinCaseCtrl=TRUE \
    >&2
    
cat ${tmpdir}/output.txt
rm -rf ${tmpdir}
"""

In [34]:
cmd = json.dumps(["bash", "-c", saige_script])
#output_df = glow.transform("pipe", input_df, cmd=cmd, input_formatter='vcf', in_vcf_header=input_vcf,
#                           output_formatter='csv', out_header='true', out_delimiter=' ')

In [35]:
print(cmd)

["bash", "-c", "#!/bin/sh\nset -e\n\nexport tmpdir=$(mktemp -d -t vcf.XXXXXX)\ncat - > ${tmpdir}/input.vcf\nbgzip -c ${tmpdir}/input.vcf > ${tmpdir}/input.vcf.gz\ntabix -p vcf ${tmpdir}/input.vcf.gz\n\nexport dataDir='/mnt/SAIGE/extdata'\n \n/opt/conda/default/bin/Rscript /mnt/SAIGE/extdata/step2_SPAtests.R \\\n    --vcfFile=${tmpdir}/input.vcf.gz \\\n    --vcfFileIndex=${tmpdir}/input.vcf.gz.tbi \\\n    --vcfField=DS \\\n    --chrom=1 \\\n    --minMAF=0.0001 \\\n    --minMAC=1 \\\n    --GMMATmodelFile=${dataDir}/output/example_binary.rda \\\n    --varianceRatioFile=${dataDir}/output/example_binary.varianceRatio.txt \\\n    --SAIGEOutputFile=${tmpdir}/output.txt \\\n    --numLinesOutput=2 \\\n    --IsOutputAFinCaseCtrl=TRUE \\\n    --IsOutputNinCaseCtrl=TRUE \\\n    --IsOutputHetHomCountsinCaseCtrl=TRUE \\\n    >&2\n    \ncat ${tmpdir}/output.txt\nrm -rf ${tmpdir}\n"]


In [13]:
input_vcf = "file:///mnt/SAIGE/extdata/input/dosage_10markers.vcf.gz"
input_df = spark.read.format("vcf").load(input_vcf)

In [36]:
output_df = glow.transform("pipe", input_df, cmd=cmd, input_formatter='vcf', in_vcf_header=input_vcf,
                          output_formatter='csv', out_header='true', out_delimiter=' ')

Py4JJavaError: An error occurred while calling z:io.projectglow.Glow.transform.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 4 times, most recent failure: Lost task 0.3 in stage 18.0 (TID 43, cluster-test-w-1.us-east4-a.c.nih-nci-dceg-episphere-dev.internal, executor 1): java.lang.IllegalStateException: Subprocess exited with status 1
	at io.projectglow.transformers.pipe.PipeIterator.hasNext(Piper.scala:203)
	at org.apache.spark.serializer.SerializationStream.writeAll(Serializer.scala:139)
	at org.apache.spark.serializer.SerializerManager.dataSerializeStream(SerializerManager.scala:174)
	at org.apache.spark.storage.BlockManager.$anonfun$doPutIterator$6(BlockManager.scala:1203)
	at org.apache.spark.storage.BlockManager.$anonfun$doPutIterator$6$adapted(BlockManager.scala:1201)
	at org.apache.spark.storage.DiskStore.put(DiskStore.scala:69)
	at org.apache.spark.storage.BlockManager.$anonfun$doPutIterator$1(BlockManager.scala:1201)
	at org

In [22]:
output_df.show()

+--------------------+
|##fileformat=VCFv4.2|
+--------------------+
|##FORMAT=<ID=DS,N...|
|##INFO=<ID=PR,Num...|
|##contig=<ID=1,le...|
| ##fileDate=20171104|
| ##source=PLINKv1.90|
|#CHROM	POS	ID	REF...|
|1	4	rs4	A	C	.	.	P...|
|1	9	rs9	A	C	.	.	P...|
|1	10	rs10	A	C	.	....|
|1	11	rs11	A	C	.	....|
|1	12	rs12	A	C	.	....|
|1	14	rs14	A	C	.	....|
|1	15	rs15	A	C	.	....|
|1	17	rs17	A	C	.	....|
|1	20	rs20	A	C	.	....|
|1	40000000	rs22	A...|
+--------------------+



In [2]:
#path = "gs://confluence_test/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz"

path = "gs://confluence_test/study/Study4_GWAS.vcf"

df = spark.read.format("vcf").load(path)

In [3]:
test_df = glow.transform('pipe', df, cmd='["/opt/conda/default/bin/bcftools", "view", "-"]',
              input_formatter = 'vcf', in_vcf_header='infer', output_formatter='vcf')

In [4]:
test_df.show()

+----------+--------+--------+--------------------+---------------+----------------+----+-------+---------------------+--------------------+
|contigName|   start|     end|               names|referenceAllele|alternateAlleles|qual|filters|splitFromMultiAllelic|           genotypes|
+----------+--------+--------+--------------------+---------------+----------------+----+-------+---------------------+--------------------+
|         5|49405662|49405663|[5:49405663:G:A,---]|              G|             [A]|null|     []|                false|[[1, [0.73, 0.255...|
|         5|49405755|49405756|[rs62390078:49405...|              C|             [A]|null|     []|                false|[[1, [0.85, 0.15,...|
|         5|49405755|49405756|[rs62390078:49405...|              C|             [T]|null|     []|                false|[[1, [0.209, 0.60...|
|         5|49405757|49405758|[rs376842142:4940...|              T|             [C]|null|     []|                false|[[1, [0.85, 0.15,...|
|         5|4